## Import Libraries

In [18]:
import importlib, sys

import vars
importlib.reload(vars)

<module 'vars' from 'c:\\GitHub\\realtor-analysis\\vars.py'>

In [ ]:
import pandas as pd
import numpy as np

## Load and View Data

In [7]:
supply = pd.read_csv(vars.supply_data_path)
print(f"Read {len(supply)} records under Supply Dataset")
print('columns:')
print(supply.columns)

demand = pd.read_csv(vars.demand_data_path)
print(f"Read {len(demand)} records under Demand Dataset")
print('columns:')
print(demand.columns)

Read 3085041 records under Supply Dataset
columns:
Index(['month_date_yyyymm', 'postal_code', 'zip_name', 'median_listing_price',
       'median_listing_price_mm', 'median_listing_price_yy',
       'active_listing_count', 'active_listing_count_mm',
       'active_listing_count_yy', 'median_days_on_market',
       'median_days_on_market_mm', 'median_days_on_market_yy',
       'new_listing_count', 'new_listing_count_mm', 'new_listing_count_yy',
       'price_increased_count', 'price_increased_count_mm',
       'price_increased_count_yy', 'price_increased_share',
       'price_increased_share_mm', 'price_increased_share_yy',
       'price_reduced_count', 'price_reduced_count_mm',
       'price_reduced_count_yy', 'price_reduced_share',
       'price_reduced_share_mm', 'price_reduced_share_yy',
       'pending_listing_count', 'pending_listing_count_mm',
       'pending_listing_count_yy', 'median_listing_price_per_square_foot',
       'median_listing_price_per_square_foot_mm',
       'median

In [8]:
supply[['postal_code', 'zip_name']]

,postal_code,zip_name
0,56662,"outing, mn"
1,24531,"chatham, va"
2,26505,"morgantown, wv"
3,23838,"chesterfield, va"
4,38115,"memphis, tn"
...,...,...
3085036,7826,"branchville, nj"
3085037,29307,"spartanburg, sc"
3085038,84620,"aurora, ut"
3085039,60517,"woodridge, il"


In [ ]:
zip2county = pd.read_excel(vars.zip2county)
print(zip2county)

print(f"Unique Zips (Supply): {supply['postal_code'].nunique()}")
print(f"Unique Zips (Demand): {demand['postal_code'].nunique()}")
print(f"Unique Zips (Crosswalk): {zip2county['zip'].nunique()}")
print(f"Unique County Codes (Crosswalk): {zip2county['county'].nunique()}")

# Check if there are any zips in supply and demand that are not in the crosswalk
supply_zips = set(supply['postal_code'].unique())
demand_zips = set(demand['postal_code'].unique())
crosswalk_zips = set(zip2county['zip'].unique())
missing_in_crosswalk = (supply_zips | demand_zips) - crosswalk_zips
if missing_in_crosswalk:
    print(f"Zips in supply/demand not in crosswalk: {len(missing_in_crosswalk)}")
else:
    print("All zips in supply and demand are covered in the crosswalk.")


         zip  county  res_ratio  bus_ratio  oth_ratio  tot_ratio
0        606   72093   1.000000   1.000000   1.000000   1.000000
1        617   72017   0.969103   0.998428   1.000000   0.971027
2        617   72054   0.030897   0.001572   0.000000   0.028973
3        674   72145   0.004759   0.017114   0.022523   0.006106
4        674   72091   0.995241   0.982886   0.977477   0.993894
...      ...     ...        ...        ...        ...        ...
53813  99688    2170   1.000000   1.000000   1.000000   1.000000
53814  99705    2090   1.000000   1.000000   1.000000   1.000000
53815  99694    2170   0.000000   0.000000   1.000000   1.000000
53816  99659    2180   0.000000   1.000000   1.000000   1.000000
53817  99680    2050   0.000000   0.000000   1.000000   1.000000

[53818 rows x 6 columns]
Unique Zips (Supply): 34104
Unique Zips (Demand): 18777
Unique Zips (Crosswalk): 39461
Unique County Codes (Crosswalk): 3227
Zips in supply/demand not in crosswalk: 33
